In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK") 

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [3]:
using JuMP
using GLPK 


Se dispone de cuatro tipos de generadores para satisfacer la demanda diaria de
electricidad de una pequeña ciudad (en mega watts). Para estimar la demanda diaria se considera
un horizonte de tiempo circular: el período 10pm - 0am del día d es seguido por el período 0am - 6am del día d + 1.

A continuación la tabla de demandas:


| Periodo          | 0am-6am | 6am-9am | 9am-12pm | 12pm-2pm | 2pm-6pm | 6pm-10pm | 10pm-0am |
|------------------|---------|---------|----------|----------|---------|----------|----------|
| **Demanda (MW)** | 12000   | 32000   | 25000    | 36000    | 25000   | 30000    | 18000    |

**Tabla 1: Demandas Estimadas**

Los generadores del mismo tipo cuentan con una capacidad máxima y pueden conectarse
a la red a partir de una cierta potencia de salida mínima. Además, tienen un costo de puesta en
marcha, un costo fijo por hora de trabajo a mínima potencia y un costo por hora por cada mega
watt generado adicional a la salida mínima. Estos datos se muestran en la siguiente tabla.



|Tipo| Generadores disponibles | Salida mínima (MW) | Máxima capacidad (MW) | Costo fijo $/h | Costo de MW adicional $/h | Costo de puesta en marcha |
|-------------------------|-------------------------|---------------------|-----------------------|----------------|---------------------------|---------------------------|
|Tipo 1| 10                | 750                 | 1750                  | 2250           | 2.7                       | 5000                      |
|Tipo 2| 4                 | 1000                | 1500                  | 1800           | 2.2                       | 1600                      |
|Tipo 3| 8                 | 1200                | 2000                  | 3750           | 1.8                       | 2400                      |
|Tipo 4| 3                 | 1800                | 3500                  | 4800           | 3.8                       | 1200                      |

**Tabla 2: Disponibilidad, Costos y Capacidades de cada tipo de generador**

Un generador puede ponerse en marcha o detenerse solo al comienzo de un período. A
diferencia de la puesta en marcha, detener un generador no tiene costo. En cualquier momento, el
grupo de generadores que se encuentre trabajando, deben poder hacer frente a un aumento del
20% de la demanda estimada. ¿Qué generadores de energía se deberían utilizar en cada período
para minimizar el costo diario total?

In [4]:

# Construcción del modelo

modelo1 = Model(GLPK.Optimizer)

# Parámetros del Problema
periodos = [
    (0, 6),
    (6, 9),
    (9, 12),
    (12, 14),
    (14, 18),
    (18, 22),
    (22, 24)
]

demanda = [12000, 32000, 25000, 36000, 25000, 30000, 18000]  

porcentajeDemanda = 1.20

tipoDeGeneradores = [1, 2, 3, 4]
generadoresDisponibles = [10, 4, 8, 3]
salidaMin = [750, 1000, 1200, 1800]
potenciaMax = [1750, 1500, 2000, 3500]
costoFijo = [2250, 1800, 3750, 4800]
costoAdicional = [2.7, 2.2, 1.8, 3.8]
costoMarcha = [5000, 1600, 2400, 1200]

# Variables de decisión
# generadores tipo t que comienzan a trabajar en el periodo p
@variable(modelo1, 0 <= x[t in 1:length(tipoDeGeneradores), p in 1:length(periodos)] <= generadoresDisponibles[t], Int)
# cantidad de generadores tipo t trabajando en el periodo p
@variable(modelo1, 0 <= y[t in 1:length(tipoDeGeneradores), p in 1:length(periodos)] <= generadoresDisponibles[t], Int)
# MWh adicionales de generadores tipo t en el periodo p
@variable(modelo1, z[t in 1:length(tipoDeGeneradores), p in 1:length(periodos)] >= 0, Int) 

#Función para obtener la cantidad de horas de un periodo p
function HorasP(p::Int64)
    periodo = periodos[p]
    return periodo[2] - periodo[1]
end

# Restricciones

# Restricción de producción anual
for t in 1:length(tipoDeGeneradores), p in 1:length(periodos)
    @constraint(modelo1, z[t,p] <= (potenciaMax[t] - salidaMin[t]) * y[t,p])
end

# Restricción de demanda
for p in 1:length(periodos)
    sumatoria = sum(salidaMin[t] * y[t,p] + z[t,p] for t in 1:length(tipoDeGeneradores))
    @constraint(modelo1, sumatoria >= demanda[p])
end

# Restricción de sobredemanda
for p in 1:length(periodos)
    sumatoria = sum(potenciaMax[t] * y[t,p] for t in 1:length(tipoDeGeneradores))
    @constraint(modelo1, sumatoria >= porcentajeDemanda * demanda[p])
end


for t in 1:length(tipoDeGeneradores)
    for p in 2:length(periodos)
        @constraint(modelo1, x[t,p] >= y[t,p] - y[t,p-1])
    end
end

for t in 1:length(tipoDeGeneradores)
    @constraint(modelo1, x[t,1] >= y[t,1] - y[t,length(periodos)])
end


# Función objetivo
@objective(modelo1, Min, sum(sum(costoMarcha[t] * x[t,p] + HorasP(p)*(costoFijo[t] * y[t,p] + costoAdicional[t] * z[t,p])  for p in 1:length(periodos)) for t in 1:length(tipoDeGeneradores)))

#Optimización
optimize!(modelo1)

#Resultados
println("Valor objetivo = ", objective_value(modelo1))

for t in 1:length(tipoDeGeneradores), p in 1:length(periodos)
    println("Valor de x[$t, $p]: ", value(x[t,p]))
end

for t in 1:length(tipoDeGeneradores), p in 1:length(periodos)
    println("Valor de y[$t, $p]: ", value(y[t,p]))
end

for t in 1:length(tipoDeGeneradores), p in 1:length(periodos)
    println("Valor de z[$t, $p]: ", value(z[t,p]))
end


Valor objetivo = 1.46581e6
Valor de x[1, 1]: 0.0
Valor de x[1, 2]: 1.0
Valor de x[1, 3]: 0.0
Valor de x[1, 4]: 3.0
Valor de x[1, 5]: 0.0
Valor de x[1, 6]: 0.0
Valor de x[1, 7]: 0.0
Valor de x[2, 1]: 0.0
Valor de x[2, 2]: 0.0
Valor de x[2, 3]: 0.0
Valor de x[2, 4]: 0.0
Valor de x[2, 5]: 0.0
Valor de x[2, 6]: 0.0
Valor de x[2, 7]: 0.0
Valor de x[3, 1]: 0.0
Valor de x[3, 2]: 6.0
Valor de x[3, 3]: 0.0
Valor de x[3, 4]: 0.0
Valor de x[3, 5]: 0.0
Valor de x[3, 6]: 0.0
Valor de x[3, 7]: 0.0
Valor de x[4, 1]: 0.0
Valor de x[4, 2]: 3.0
Valor de x[4, 3]: 0.0
Valor de x[4, 4]: 2.0
Valor de x[4, 5]: 0.0
Valor de x[4, 6]: 2.0
Valor de x[4, 7]: 0.0
Valor de y[1, 1]: 3.0
Valor de y[1, 2]: 4.0
Valor de y[1, 3]: 4.0
Valor de y[1, 4]: 7.0
Valor de y[1, 5]: 3.0
Valor de y[1, 6]: 3.0
Valor de y[1, 7]: 3.0
Valor de y[2, 1]: 4.0
Valor de y[2, 2]: 4.0
Valor de y[2, 3]: 4.0
Valor de y[2, 4]: 4.0
Valor de y[2, 5]: 4.0
Valor de y[2, 6]: 4.0
Valor de y[2, 7]: 4.0
Valor de y[3, 1]: 2.0
Valor de y[3, 2]: 8.0
Valor

Modelo:

Definiciones:

TAREAS = {1,.....,N} conjunto de tareas, con la n-esima tarea ficticia final

DURACION_{i}: duracion de la tarea i

Variables de decisión:

$x_{i}:$ tiempo de inicio de la tarea i. Con i = 1,...,18, con i perteneciente a TAREAS

Función objetivo: (minimizar costos)

$$ minimizar \ x_{n} $$ 

Restricciones:

$$ x_{i} \ >= \ 0  $$ 

$$ G(TAREAS, ARCOS) \ grafo \ dirigido $$ 

$$ ARCOS \ =  \ {(i,j) \ pertenecen \ TAREASxTAREAS: \ si \ tarea \ i \ precede \ a \ la \ tarea \ j} $$ 

$$ para \ todo \ (i,j) \ pertenecientes \ a \ ARCOS: \ x_{i} + DURACION_{i} \leq x_{j} $$ 

In [5]:
# Grafo como matriz de adyacencia la cual es 19 x 19, creando una tarea 19 ficticia que depende de todas las tareas anteriores
#para corroborar que el tiempo final sea el correcto

G = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #1
     1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #2
     0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #3
     0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #4
     0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #5
     0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #6
     0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #7
     0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; #8
     0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; #9
     0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #10
     0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; #11
     0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0; #12
     0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0; #13
     0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; #14
     0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0; #15
     0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0; #16
     0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0; #17
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0; #18
     1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] #19

DURACION = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1 0] # variable de la duracion de cada tarea (tarea ficticia con duracion 0)

# CONSTRUCCIÓN DEL MODELO

modelo2 = Model(GLPK.Optimizer)

# VARIABLES DE DECISIÓN
@variable(modelo2, x[1:19] >= 0) # tiempo de inicio de la tarea i. Con i = 1,...,19

# FUNCIÓN OBJETIVO

# el objetivo es minimizar el tiempo para empezar ( el resultado que da seria terminar), todas las tareas
@objective(modelo2, Min, x[19] ) 

# RESTRICCIONES
# Se Recorre el grafo de dependencias
for i in 1:19
    for j in 1:19
        if G[i, j] == 1
        # Se aplican las dependencias de las tareas        
            @constraint(modelo2, (x[j] + DURACION[j] <= x[i]))
            
            
        end
    end
end


# SOLUCIÖN DEL MODELO
optimize!(modelo2)
println("Valor objetivo = ", objective_value(modelo2))
for i in 1:19
    println("Variable X",i," = ",value(x[i]))
end

Valor objetivo = 64.0
Variable X1 = 0.0
Variable X2 = 2.0
Variable X3 = 18.0
Variable X4 = 18.0
Variable X5 = 27.0
Variable X6 = 37.0
Variable X7 = 26.0
Variable X8 = 43.0
Variable X9 = 43.0
Variable X10 = 26.0
Variable X11 = 43.0
Variable X12 = 52.0
Variable X13 = 28.0
Variable X14 = 18.0
Variable X15 = 26.0
Variable X16 = 46.0
Variable X17 = 54.0
Variable X18 = 63.0
Variable X19 = 64.0


Ejercicio 3

Para aumentar su competitividad, una compañía desea optimizar la manufactura de sus productos más vendidos. La principal actividad de la compañía es la producción de tarjetas electrónicas. La compañía, también produce los componentes para el ensamblado de las tarjetas.
Una buena planificación de la producción de los componentes constituye un factor de éxito decisivo para la empresa. La demanda de los componentes en este caso es interna, por lo tanto, fácil de proyectar.

Se requiere planificar para los próximos seis meses la producción de cuatro componentes referenciados como AX22-M1, AX22-M2, CX32-N1 y CX32-N2.

La demanda mensual para cada tipo de componente, los costos de producción y almacenamiento, y los niveles deseados de stock inicial y final para cada producto se listan en la tabla nro. 4.

¿Cuál es el plan de producción que minimiza la suma de los costos de producción y almacenamiento?

|                  Demanda        |      Costos         | Niveles de Stock|
|----------------------------------------------------------|---------------------|----------------|
| Mes             | Producción  Bodega | Inicial Final |

***Tabla 3: Separación de los componentes***


| Mes            | 1    | 2    | 3    | 4    | 5    | 6    | Producción | Bodega | Inicial | Final |
|----------------|------|------|------|------|------|------|------------|--------|---------|-------|
| AX22-M1        | 1500 | 3000 | 2000 | 4000 | 2000 | 2500 | 20         | 0.4    | 10      | 50    |
| AX22-M2        | 1300 | 800  | 800  | 1000 | 1100 | 900  | 25         | 0.5    | 0       | 10    |
| CX32-N1        | 2200 | 1500 | 2900 | 1800 | 1200 | 2100 | 10         | 0.3    | 0       | 10    |
| CX32-N2        | 1400 | 1600 | 1500 | 1000 | 1100 | 1200 | 15         | 0.3    | 0       | 10    |



***Tabla 4: Demanda de los componentes***


In [6]:
# Construcción del modelo

modelo3 = Model(GLPK.Optimizer)

# Parámetros del Problema

meses = [1, 2, 3, 4, 5, 6]
demandas = [[1500, 3000, 2000, 4000, 2000, 2500], 
            [1300,  800,  800, 1000, 1100,  900],
            [2200, 1500, 2900, 1800, 1200, 2100],
            [1400, 1600, 1500, 1000, 1100, 1200]]
componentes = ["AX22-M1", "AX22-M2", "CX32-N1",  "CX32-N2"]
costosProduccion = [20, 25, 10, 15] 
costosBodega = [0.4, 0.5, 0.3, 0.3]
stockInicial = [10, 0, 0, 0]  
stockFinal = [50, 10, 10, 10]

# variable de decisión 
@variable(modelo3, x[c in 1:length(componentes), m in meses] >= 0, Int)
@variable(modelo3, y[c in 1:length(componentes), m in meses] >= 0, Int)

# Restricciones 

for c in 1:length(componentes)
  for m in 1:length(meses)
    @constraint(modelo3, x[c, m] + stockInicial[c] >= demandas[c][m])
    if(m == 1)
      @constraint(modelo3, y[c, m] == x[c, m] - demandas[c][m] +  stockInicial[c])
    end
    if(m > 1)
      @constraint(modelo3, y[c, m] ==  x[c, m] - demandas[c][m] + y[c, m-1]) 
    end
    @constraint(modelo3, y[c, 6] == stockFinal[c])
  end
end


#Función objetivo minimiza la suma de los costos de producción y almacenamiento

@objective(modelo3, Min,
    sum(costosProduccion[c] * x[c, m] # costo de producción por la producción
          + costosBodega[c] * y[c, m] # por el costo de bodega por la cantidad en bodega
    for c in 1:length(componentes), m in meses)
)

optimize!(modelo3)

println("Valor objetivo = ", objective_value(modelo3))


for c in 1:length(componentes), m in 1:length(meses)
    println( "Mes: ", meses[m], " Componente: ", componentes[c], " Cantidad: ", value(x[c,m])," Cantidad en bodega: ", value(y[c,m]))
end

Valor objetivo = 682831.0
Mes: 1 Componente: AX22-M1 Cantidad: 1490.0 Cantidad en bodega: 0.0
Mes: 2 Componente: AX22-M1 Cantidad: 3000.0 Cantidad en bodega: 0.0
Mes: 3 Componente: AX22-M1 Cantidad: 2000.0 Cantidad en bodega: 0.0
Mes: 4 Componente: AX22-M1 Cantidad: 4000.0 Cantidad en bodega: 0.0
Mes: 5 Componente: AX22-M1 Cantidad: 2000.0 Cantidad en bodega: 0.0
Mes: 6 Componente: AX22-M1 Cantidad: 2550.0 Cantidad en bodega: 50.0
Mes: 1 Componente: AX22-M2 Cantidad: 1300.0 Cantidad en bodega: 0.0
Mes: 2 Componente: AX22-M2 Cantidad: 800.0 Cantidad en bodega: 0.0
Mes: 3 Componente: AX22-M2 Cantidad: 800.0 Cantidad en bodega: 0.0
Mes: 4 Componente: AX22-M2 Cantidad: 1000.0 Cantidad en bodega: 0.0
Mes: 5 Componente: AX22-M2 Cantidad: 1100.0 Cantidad en bodega: 0.0
Mes: 6 Componente: AX22-M2 Cantidad: 910.0 Cantidad en bodega: 10.0
Mes: 1 Componente: CX32-N1 Cantidad: 2200.0 Cantidad en bodega: 0.0
Mes: 2 Componente: CX32-N1 Cantidad: 1500.0 Cantidad en bodega: 0.0
Mes: 3 Componente: CX32